In [44]:
import os
import rdflib
import pandas as pd

# Loading RDF Datasets

In [33]:
def get_rdf_filenames(dirname: str):
    """  """
    rdf_filenames = []
    for filename in os.listdir(dirname):
        file_path = os.path.join(dirname, filename)
        if os.path.isfile(file_path) and filename.split('.')[-1] == 'ttl':
            rdf_filenames.append(filename)
    return rdf_filenames

def load_rdf_files(dirnames: list):
    """  """
    g = rdflib.Graph()

    for dirname in dirnames:
        print(f'Load RDF data from directory {dirname} into RDF graph.')
    
        rdf_filenames = get_rdf_filenames(dirname)
        for rdf_filename in rdf_filenames:
            g.parse(f'{dirname}/{rdf_filename}')

    return g

In [34]:
rdf_dirnames = ['HAMLET_DATA', 'MONARCH_PHENOPACKET_STORE_DATA']
g = load_rdf_files(rdf_dirnames)

Load RDF data from directory HAMLET_DATA into RDF graph.
Load RDF data from directory MONARCH_PHENOPACKET_STORE_DATA into RDF graph.


# SPARQL Queries

In [39]:
SCENARIO_NR = 4

with open(f'SPARQL/scenario_{SCENARIO_NR}/question1_a.rq', 'r') as file:
    query = file.read()

In [42]:
results = g.query(query)

In [43]:
genes = []

for result_row in results:
    genes.append({'id': result_row[0], 'symbol': result_row[1]})

In [45]:
genes_df = pd.DataFrame(genes)
genes_df

,id,symbol
0,ENSG00000213281,NRAS
1,ENSG00000141510,TP53


In [46]:
from myst_nb import glue

glue('scenario4_genes', genes_df)

,id,symbol
0,ENSG00000213281,NRAS
1,ENSG00000141510,TP53
